In [1]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates


from QUANTAXIS.QAUtil import DATABASE
from QUANTAXIS.QAUtil import  trade_date_sse
from QUANTAXIS.QAUtil.QADate_trade import (
    QA_util_get_pre_trade_date,
    QA_util_get_next_trade_date,
    QA_util_if_tradetime
)

import ind.alpha191 as a191
from scipy import stats
import statsmodels.api as sm

# import cpuinfo
# if 'ntel' in cpuinfo.get_cpu_info()['brand_raw']:
# from sklearnex import patch_sklearn, unpatch_sklearn
# unpatch_sklearn() ##注意，少量数据的线性回归没有优势。慎用，存在内存泄露

from sklearn import linear_model


%load_ext autoreload
%autoreload 2
%aimport ind.alpha191

def pretreate_data(data):
    returns = smpl.get_current_return(data,'close')
    returns.name = 'returns'
    ret_forward = smpl.get_forward_return(data,'close')
    ret_forward.name = 'ret_forward'
    # {'Open', 'cap', 'close', 'high', 'ind', 'low', 'returns', 'volume', 'vwap'}
    data = pd.concat([data, returns, ret_forward], axis=1)
    data = data.assign(vwap=data.amount/(data.volume*100))
    data = smpl.add_marketvalue_industry(data)
    data.rename(columns = {"open":"Open",'totalCapital':'cap','industry':'ind'}, inplace=True)
    data = smpl.add_report_inds(data,'netAssetsPerShare')
    data['pb']=data['close']/data['netAssetsPerShare']
    
    df_ids = data.index.get_level_values(0)
    
    benchmark = smpl.get_benchmark('沪深300', start=df_ids.min(), end=df_ids.max()).data
    benchmark = benchmark[['open','close']].reset_index('code',drop=True)
    data['bm_index_open'] = df_ids.map(benchmark['open'])
    data['bm_index_close'] = df_ids.map(benchmark['close'])
    return data
    # return pd.concat([data, close_ind, vwap_ind, low_ind, high_ind, volume_ind, adv20_ind, adv40_ind, adv81_ind, co_mixed_ind, oh_mixed_ind, lv_mixed_ind], axis=1)

def assemble_stocks_by_codes(codes):
    files = list(map(lambda x:x+'_train_qfq',codes))
    return pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),files))).sort_index(level=0)


In [2]:
l = smpl.get_codes_from_blockname('沪深300', sse='all')
df_all =  assemble_stocks_by_codes(l)
df_treated = pretreate_data(df_all)

In [3]:
# ids = [11,24,38,41,42,47,57,69,80,82,83,88,93,97]
# ids = [92,157,149,191]
# ['000001','601728','000002','601825', '601868']

ids = np.arange(1,192)
np.random.shuffle(ids)

def generate_alpha_factors(fun_ids, df=None):
    import ind.alpha191 as a191
    from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
    from base.JuUnits import excute_for_multidates
    import numpy as np

    for i in fun_ids:
    # for i in ids:
        fun_name = 'alpha'+ '0'*(3- int(np.log10(i)+1)) +str(i)
        if fun_name != 'alpha030':
            indx = excute_for_multidates(df, lambda x: getattr(a191, fun_name)(x) ,level=1)
        else:
            indx = getattr(a191, fun_name)(df)
        # indx = excute_for_multidates(df_treated.loc[(slice(None),['000001','601728','000002','601825', '601868']),:], lambda x: getattr(a191, fun_name)(x) ,level=1)
        indx.name = '191_'+fun_name
        save_cache('{}_train'.format(indx.name), indx, cache_type=CACHE_TYPE.FACTOR)
        # print(indx)

worker=6
task = task_chunk_split(ids, worker)
results = parallal_task(worker, generate_alpha_factors, task, df=df_treated)


Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:23376  pid:25024


  0%|          | 0/6 [00:00<?, ?it/s]